In [11]:
import pandas as pd
from tqdm import tqdm

path = "/gscratch/cse/lee0618/cifar-100/meta/pairs.csv"
df = pd.read_csv(path)

md_path = "/gscratch/cse/lee0618/cifar-100/meta/train.csv"
md = pd.read_csv(md_path)

print(len(df))
superclass_agreement = df.loc[df["superclass_1"] == df["superclass_2"]]
label_agreement = df.loc[df["label_1"] == df["label_2"]]
print(len(superclass_agreement) / len(df)) # 0.73 (k = 4), 0.739 (k = 3), 0.744 (k = 2), 0.753 (k = 1)
print(len(label_agreement) / len(df)) # 0.62 (k = 4), 0.625 (k = 3), 0.63 (k = 2), 0.643 (k = 1)

print(df.loc[df["superclass_1"] != df["superclass_2"]].head())

70658
0.7442752413031787
0.63193127459028
    Unnamed: 0      id_1        id_2  label_1  label_2  superclass_1  \
1            1  19_1.jpg  41_337.jpg       19       41            11   
17          17  23_1.jpg  69_283.jpg       23       69            10   
26          26  17_1.jpg  37_354.jpg       17       37            11   
30          30  39_2.jpg   9_220.jpg       39        9             5   
31          31  39_2.jpg  16_234.jpg       39       16             5   

    superclass_2  
1             19  
17            19  
26             9  
30             8  
31             3  
